In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
import urllib.request
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep
import random
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from tqdm import tqdm

In [2]:
from Main_function import get_Data_On_Page, get_Data_OneProduct_Link 

path = 'chromedriver.exe'
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-notifications')
chrome_options.add_argument('--disable-infobars')

service = Service(executable_path=path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [3]:
#Empty list, to store data from all pages
AllData = []

for page in tqdm(range(16, 26), desc="Crawling pages"):
    main_link = f'https://www.lazada.vn/catalog/?page={page}&q=laptop'
    driver.get(main_link)
    sleep(random.uniform(2,5))                             #Random sleep to mimic human behavior and avoid getting blocked

    df_Laptop = get_Data_On_Page(driver)
    AllData.append(df_Laptop)

Crawling pages: 100%|██████████| 10/10 [04:50<00:00, 29.02s/it]


In [4]:
#Combine into a DataFrame
Data_Laptop_Page16_To_Page25 = pd.concat(AllData, ignore_index=True)
Data_Laptop_Page16_To_Page25.index = np.arange(1, len(Data_Laptop_Page16_To_Page25)+1)
Data_Laptop_Page16_To_Page25

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location
1,laptop,"Dell Inspiron 5420 {Core i5-1240P, RAM 16G, SS...",https://www.lazada.vn/products/dell-inspiron-5...,2.995.000 ₫,50% Off,None,None,Hà Nội
2,laptop,"Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...",https://www.lazada.vn/products/may-tinh-xach-t...,39.000 ₫,17% Off,313 Đã bán,(67),Hồ Chí Minh
3,laptop,( E- Voucher ) Sở Hữu I.phone 15 Plus 512Gb - ...,https://www.lazada.vn/products/e-voucher-so-hu...,29.000 ₫,42% Off,184 Đã bán,(78),Hồ Chí Minh
4,laptop,Laptop Toshiba Dynabook B35 màn 15.6 inch - Co...,https://www.lazada.vn/products/laptop-toshiba-...,1.727.600 ₫,,None,None,Hà Nội
5,laptop,Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...,https://www.lazada.vn/products/laptop-dell-ins...,3.200.000 ₫,50% Off,None,None,Hà Nội
...,...,...,...,...,...,...,...,...
396,laptop,Laptop HP 15s-Fq5231TU ( 8U241PA ) Natural Sil...,https://www.lazada.vn/products/laptop-hp-15s-f...,10.590.000 ₫,15% Off,None,None,Hà Nội
397,laptop,Laptop Mini 12.5in Dell 7270 Core I7 - 6600u/R...,https://www.lazada.vn/products/laptop-mini-125...,4.790.000 ₫,19% Off,5 Đã bán,(1),Hồ Chí Minh
398,laptop,"Laptop 2 trong 1 Fujitsu Q737 Core i7-7600U, 8...",https://www.lazada.vn/products/laptop-2-trong-...,3.400.000 ₫,,12 Đã bán,(5),Hưng Yên
399,laptop,"Máy tính cũ Macboook,, Pro rentina 2015 13in C...",https://www.lazada.vn/products/may-tinh-cu-mac...,6.250.000 ₫,4% Off,None,None,Hồ Chí Minh


In [5]:
#Initialize dictionary to store details for all products
Products = {
    "Price_original": [],
    "Ship_price": [],
    "Return": [],
    "Sale_rating": [],
    "Ship_on_time": [],
    "Chat_response": [],
    "One_star": [],
    "Two_star": [],
    "Three_star": [],
    "Four_star": [],
    "Five_star": []
}

#Create a progress bar to track the crawl process
for link in tqdm(Data_Laptop_Page16_To_Page25.Link, desc="Crawling link product"):
    LinkProduct = get_Data_OneProduct_Link(driver, link)
    for key in Products.keys():
        Products[key].append(LinkProduct[key])

Crawling link product: 100%|██████████| 400/400 [5:05:57<00:00, 45.89s/it]    


In [6]:
#Add columns to combined_data
for key, value in Products.items():
    Data_Laptop_Page16_To_Page25[key] = value

In [7]:
# Convert to csv
Data_Laptop_Page16_To_Page25.to_csv("Data_Laptop_Page16_To_Page25.csv", index=False)
Data_Laptop_Page16_To_Page25

,Type,Title,Link,Price_sale,Sale_off,Total_sold,Preview,Location,Price_original,Ship_price,Return,Sale_rating,Ship_on_time,Chat_response,One_star,Two_star,Three_star,Four_star,Five_star
1,laptop,"Dell Inspiron 5420 {Core i5-1240P, RAM 16G, SS...",https://www.lazada.vn/products/dell-inspiron-5...,2.995.000 ₫,50% Off,None,None,Hà Nội,None,None,None,None,None,None,[0],[0],[0],[0],[0]
2,laptop,"Máy Tính Xách Tay Bàn Phím Bao Gồm, Màng Bảo V...",https://www.lazada.vn/products/may-tinh-xach-t...,39.000 ₫,17% Off,313 Đã bán,(67),Hồ Chí Minh,47.000 ₫,17.000 ₫,None,96%,98%,97%,[11],[1],[3],[5],[47]
3,laptop,( E- Voucher ) Sở Hữu I.phone 15 Plus 512Gb - ...,https://www.lazada.vn/products/e-voucher-so-hu...,29.000 ₫,42% Off,184 Đã bán,(78),Hồ Chí Minh,50.000 ₫,16.500 ₫,None,Nhà bán hàng mới,100%,71%,[7],[2],[1],[1],[68]
4,laptop,Laptop Toshiba Dynabook B35 màn 15.6 inch - Co...,https://www.lazada.vn/products/laptop-toshiba-...,1.727.600 ₫,,None,None,Hà Nội,None,Miễn phí,None,Nhà bán hàng mới,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[0]
5,laptop,Laptop Dell Inspiron 7567 máy tính giá rẻ(Core...,https://www.lazada.vn/products/laptop-dell-ins...,3.200.000 ₫,50% Off,None,None,Hà Nội,6.400.000 ₫,Miễn phí,None,Không đủ thông tin,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[0]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,laptop,Laptop HP 15s-Fq5231TU ( 8U241PA ) Natural Sil...,https://www.lazada.vn/products/laptop-hp-15s-f...,10.590.000 ₫,15% Off,None,None,Hà Nội,12.460.000 ₫,115.100 ₫,30 Ngày Trả Hàng Miễn Phí,92%,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[0]
397,laptop,Laptop Mini 12.5in Dell 7270 Core I7 - 6600u/R...,https://www.lazada.vn/products/laptop-mini-125...,4.790.000 ₫,19% Off,5 Đã bán,(1),Hồ Chí Minh,5.900.000 ₫,Miễn phí,None,90%,Không đủ thông tin,Không đủ thông tin,[0],[0],[0],[0],[1]
398,laptop,"Laptop 2 trong 1 Fujitsu Q737 Core i7-7600U, 8...",https://www.lazada.vn/products/laptop-2-trong-...,3.400.000 ₫,,12 Đã bán,(5),Hưng Yên,None,70.000 ₫,None,91%,65%,100%,[0],[0],[0],[0],[5]
399,laptop,"Máy tính cũ Macboook,, Pro rentina 2015 13in C...",https://www.lazada.vn/products/may-tinh-cu-mac...,6.250.000 ₫,4% Off,None,None,Hồ Chí Minh,8.500.000 ₫,Miễn phí,None,97%,100%,100%,[0],[0],[0],[0],[0]
